In [426]:
import os
import cv2
import time
import joblib
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, LSTM, Dense, Dropout, MaxPool2D, Flatten

from sklearn.utils import shuffle
from sklearn.metrics import classification_report, accuracy_score, precision_score

tf.__version__

'2.4.1'

In [427]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [428]:
# -------- TEST USER ----------- #

TEST_USER      = '001'

# BASE_DIR       = '../'

# Google Drive
BASE_DIR       = '/content/drive/MyDrive/Research/Hand Gesture/GitHub/'

IMG_DIR        = 'BW-Spatial-Path-Images/'
LOG_DIR        = 'Logs/'

USERS          = ['001', '002', '003', '004', '005', '006', '007']

# ------------------------------- Only Dynalic Gestures ------------------------------ #
GESTURES       = ['j', 'z', 'bad', 'deaf', 'fine', 'good', 'goodbye', 'hello', 'hungry',
                  'me', 'no', 'please', 'sorry', 'thankyou', 'yes', 'you']

PLANES         = ['XY', 'YZ', 'ZX']

BATCH_SIZE     = 32
IMG_LEN        = 40
IMG_SIZE       = (IMG_LEN, IMG_LEN)

# ------------- FOR THE GREATER GOOD :) ------------- #
DATASET_LEN    = 1120
TRAIN_LEN      = 960
TEST_LEN       = 160

EPOCHS         = 30
LEARNING_RATE  = 0.001
DECAY          = 0.00001

C1             = 32
C2             = 32
D1             = 256
DO             = 0.3

CONFIG         = 'BW_L7_' + str(IMG_SIZE) + '_C1' + str(C1) + '_C2' + str(C2) + '_D1' + \
                 str(D1) + '_DO' + str(DO) + '_E' + str(EPOCHS) + '_LR' + str(LEARNING_RATE) 

XY_WEIGHTS     = np.array([0.91, 0.75, 0.61, 0.63, 0.51, 0.66, 0.81, 0.65, 0.65, 0.31,
                           0.66, 0.29, 0.34, 0.64, 0.64, 0.31])
YZ_WEIGHTS     = np.array([0.73, 0.71, 0.70, 0.79, 0.76, 0.38, 0.80, 0.61, 0.58, 0.73,
                           0.49, 0.26, 0.26, 0.52, 0.59, 0.54])
ZX_WEIGHTS     = np.array([0.33, 0.66, 0.51, 0.54, 0.37, 0.51, 0.71, 0.30, 0.75, 0.41,
                           0.40, 0.27, 0.24, 0.61, 0.36, 0.49])

In [429]:
def load_data(plane):
    X_train = np.zeros((TRAIN_LEN, IMG_LEN, IMG_LEN, 1))
    X_test = np.zeros((TEST_LEN, IMG_LEN, IMG_LEN, 1))
    y_train = np.zeros((TRAIN_LEN, 1))
    y_test = np.zeros((TEST_LEN, 1))
    
    train_count = 0
    test_count = 0
        
    for gesture in GESTURES:
        print('loading data for ' + gesture + ' gesture on the ' + plane + ' plane ... ', end='')
        path = os.path.join(BASE_DIR, IMG_DIR, plane, gesture)
        for filename in os.listdir(path):
            img = cv2.imread(os.path.join(path, filename), cv2.IMREAD_GRAYSCALE)
            resized = cv2.resize(img, IMG_SIZE)
            if filename[1:4] != TEST_USER:
                X_train[train_count, :] = np.expand_dims(resized, axis=-1)
                y_train[train_count, 0] = GESTURES.index(gesture)
                train_count = train_count + 1
            else:
                X_test[test_count, :] = np.expand_dims(resized, axis=-1)
                y_test[test_count, 0] = GESTURES.index(gesture)
                test_count = test_count + 1
                
        print('√')
        
    return X_train, X_test, y_train, y_test

def load_and_save_data(plane):
    X = np.zeros((DATASET_LEN, IMG_LEN, IMG_LEN, 1))
    y = np.zeros((DATASET_LEN, 1))
    
    train_count = 0
    test_count  = TRAIN_LEN
        
    for gesture in GESTURES:
        print('loading data for ' + gesture + ' gesture on the ' + plane + ' plane ... ', end='')
        path = os.path.join(BASE_DIR, IMG_DIR, plane, gesture)
        for filename in os.listdir(path):
            img = cv2.imread(os.path.join(path, filename), cv2.IMREAD_GRAYSCALE)
            resized = cv2.resize(img, IMG_SIZE)
            if filename[1:4] != TEST_USER:
                X[train_count, :] = np.expand_dims(resized, axis=-1)
                y[train_count, 0] = GESTURES.index(gesture)
                train_count = train_count + 1
            else:
                X[test_count, :] = np.expand_dims(resized, axis=-1)
                y[test_count, 0] = GESTURES.index(gesture)
                test_count = test_count + 1
                
        print('√')

    joblib.dump(X, BASE_DIR + 'X_BW_' + plane + str(IMG_SIZE) + '.joblib')
    joblib.dump(y, BASE_DIR + 'Y_BW_' + plane + str(IMG_SIZE) + '.joblib')

def load_data_from_joblib(plane):
    print('Loading data for ' + plane + ' plane ... ', end='')
    X = joblib.load(BASE_DIR + 'X_BW_' + plane + str(IMG_SIZE) + '.joblib')
    y = joblib.load(BASE_DIR + 'Y_BW_' + plane + str(IMG_SIZE) + '.joblib')
    test_user = int(TEST_USER)
    X_train = X[:TRAIN_LEN, :, :, :]
    y_train = y[:TRAIN_LEN, :]
    X_test = X[TRAIN_LEN:, :, :, :]
    y_test = y[TRAIN_LEN:, :]

    print('√')

    return X_train, X_test, y_train, y_test

In [430]:
# X_train_xy, X_test_xy, y_train_xy, y_test_xy = load_data('XY')
# X_train_yz, X_test_yz, y_train_yz, y_test_yz = load_data('YZ')
# X_train_zx, X_test_zx, y_train_zx, y_test_zx = load_data('ZX')

# Save to Google  Drive
# load_and_save_data('XY')
# load_and_save_data('YZ')
# load_and_save_data('ZX')

# # Load from Google Drive
X_train_xy, X_test_xy, y_train_xy, y_test_xy = load_data_from_joblib('XY')
X_train_yz, X_test_yz, y_train_yz, y_test_yz = load_data_from_joblib('YZ')
X_train_zx, X_test_zx, y_train_zx, y_test_zx = load_data_from_joblib('ZX')

Loading data for XY plane ... √
Loading data for YZ plane ... √
Loading data for ZX plane ... √


In [431]:
IMG_SHAPE = IMG_SIZE + (1,)
rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./255.0, offset= 0, input_shape=IMG_SHAPE)

In [432]:
def get_model():
    model = Sequential()

    model.add(rescale)

    model.add(Conv2D(C1, (3, 3), activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(DO))

    model.add(Conv2D(C2, (3, 3), activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(DO))

    model.add(Flatten())
    model.add(Dense(D1, activation='relu'))
    model.add(Dropout(DO))

    model.add(Dense(len(GESTURES)))
    
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=LEARNING_RATE, decay=DECAY),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    
    return model

In [433]:
# !pip install -q -U keras-tuner

# import kerastuner as kt

# def model_builder(hp):
#     model = Sequential()
#     model.add(rescale)
#     model.add(Flatten())

#     # Tune the number of units in the first Dense layer
#     # Choose an optimal value between 32-512
#     hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
#     model.add(Dense(units=hp_units, activation='relu'))
#     model.add(Dense(16))

#     # Tune the learning rate for the optimizer
#     # Choose an optimal value from 0.01, 0.001, or 0.0001
#     hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

#     model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
#                 loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#                 metrics=['accuracy'])

#     return model

# tuner = kt.Hyperband(model_builder,
#                     objective='val_accuracy',
#                     max_epochs=10,
#                     factor=3,
#                     directory='my_dir',
#                     project_name='hand_gesture')
  
# stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# tuner.search(X_train_xy, y_train_xy, epochs=50, validation_data=(X_test_xy, y_test_xy), callbacks=[stop_early])

# # Get the optimal hyperparameters
# best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

# print(f"""
# The hyperparameter search is complete. The optimal number of units in the first densely-connected
# layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
# is {best_hps.get('learning_rate')}.
# """)

In [434]:
model_xy = get_model()
X_train_xy, y_train_xy = shuffle(X_train_xy, y_train_xy)
history_xy = model_xy.fit(X_train_xy, y_train_xy, validation_data=(X_test_xy, y_test_xy), epochs=EPOCHS)

Epoch 1/30
30/30 [==============================] - 1s 15ms/step - loss: 2.7896 - accuracy: 0.0647 - val_loss: 2.7179 - val_accuracy: 0.2125
Epoch 2/30
30/30 [==============================] - 0s 9ms/step - loss: 2.5521 - accuracy: 0.2064 - val_loss: 2.2379 - val_accuracy: 0.4437
Epoch 3/30
30/30 [==============================] - 0s 9ms/step - loss: 1.9773 - accuracy: 0.3590 - val_loss: 2.0180 - val_accuracy: 0.4000
Epoch 4/30
30/30 [==============================] - 0s 9ms/step - loss: 1.4902 - accuracy: 0.5529 - val_loss: 1.8556 - val_accuracy: 0.4563
Epoch 5/30
30/30 [==============================] - 0s 10ms/step - loss: 1.2143 - accuracy: 0.6216 - val_loss: 1.8546 - val_accuracy: 0.4250
Epoch 6/30
30/30 [==============================] - 0s 9ms/step - loss: 0.9928 - accuracy: 0.6666 - val_loss: 1.7892 - val_accuracy: 0.4187
Epoch 7/30
30/30 [==============================] - 0s 9ms/step - loss: 0.8182 - accuracy: 0.7320 - val_loss: 1.8707 - val_accuracy: 0.4375
Epoch 8/30
30/30 [

In [435]:
# prob_xy = tf.keras.Sequential([model_xy, tf.keras.layers.Softmax()])
# y_pred_xy = prob_xy.predict(X_test_xy)
y_pred_xy = model_xy.predict(X_test_xy)
y_pred = np.argmax(y_pred_xy, axis=1)
print(classification_report(y_test_xy.ravel(), y_pred, zero_division=0))
prc_xy = precision_score(y_test_xy.ravel(), y_pred, zero_division=0, average=None)
tf.keras.backend.clear_session()

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        10
         1.0       1.00      0.10      0.18        10
         2.0       0.25      0.10      0.14        10
         3.0       0.50      0.60      0.55        10
         4.0       0.60      0.90      0.72        10
         5.0       0.00      0.00      0.00        10
         6.0       0.53      0.90      0.67        10
         7.0       0.67      0.60      0.63        10
         8.0       0.60      0.90      0.72        10
         9.0       0.21      0.60      0.32        10
        10.0       0.67      0.60      0.63        10
        11.0       0.00      0.00      0.00        10
        12.0       0.00      0.00      0.00        10
        13.0       0.40      0.20      0.27        10
        14.0       0.33      0.80      0.47        10
        15.0       0.50      0.10      0.17        10

    accuracy                           0.46       160
   macro avg       0.45   

In [436]:
model_yz = get_model()
X_train_yz, y_train_yz = shuffle(X_train_yz, y_train_yz)
history_yz = model_yz.fit(X_train_yz, y_train_yz, validation_data=(X_test_yz, y_test_yz), epochs=EPOCHS)

Epoch 1/30
30/30 [==============================] - 1s 15ms/step - loss: 2.8520 - accuracy: 0.0767 - val_loss: 2.7320 - val_accuracy: 0.1688
Epoch 2/30
30/30 [==============================] - 0s 9ms/step - loss: 2.6030 - accuracy: 0.2135 - val_loss: 2.0515 - val_accuracy: 0.2500
Epoch 3/30
30/30 [==============================] - 0s 9ms/step - loss: 1.9048 - accuracy: 0.3643 - val_loss: 1.7648 - val_accuracy: 0.4375
Epoch 4/30
30/30 [==============================] - 0s 9ms/step - loss: 1.4131 - accuracy: 0.5391 - val_loss: 1.6186 - val_accuracy: 0.5250
Epoch 5/30
30/30 [==============================] - 0s 9ms/step - loss: 1.2830 - accuracy: 0.5820 - val_loss: 1.5680 - val_accuracy: 0.5562
Epoch 6/30
30/30 [==============================] - 0s 9ms/step - loss: 1.0312 - accuracy: 0.6555 - val_loss: 1.4815 - val_accuracy: 0.5500
Epoch 7/30
30/30 [==============================] - 0s 9ms/step - loss: 0.9248 - accuracy: 0.6918 - val_loss: 1.4869 - val_accuracy: 0.5437
Epoch 8/30
30/30 [=

In [437]:
# prob_yz = tf.keras.Sequential([model_yz, tf.keras.layers.Softmax()])
# y_pred_yz = prob_yz.predict(X_test_yz)
y_pred_yz = model_yz.predict(X_test_yz)
y_pred = np.argmax(y_pred_yz, axis=1)
print(classification_report(y_test_yz.ravel(), y_pred, zero_division=0))
prc_yz = precision_score(y_test_yz.ravel(), y_pred, zero_division=0, average=None)
tf.keras.backend.clear_session()

              precision    recall  f1-score   support

         0.0       1.00      0.90      0.95        10
         1.0       1.00      0.30      0.46        10
         2.0       0.33      0.30      0.32        10
         3.0       1.00      0.90      0.95        10
         4.0       0.90      0.90      0.90        10
         5.0       1.00      0.30      0.46        10
         6.0       0.83      0.50      0.62        10
         7.0       0.60      0.90      0.72        10
         8.0       0.77      1.00      0.87        10
         9.0       0.91      1.00      0.95        10
        10.0       0.00      0.00      0.00        10
        11.0       0.14      0.20      0.17        10
        12.0       0.60      0.30      0.40        10
        13.0       0.57      0.80      0.67        10
        14.0       0.75      0.30      0.43        10
        15.0       0.25      0.80      0.38        10

    accuracy                           0.59       160
   macro avg       0.67   

In [438]:
model_zx = get_model()
X_train_zx, y_train_zx = shuffle(X_train_zx, y_train_zx)
history_zx = model_zx.fit(X_train_zx, y_train_zx, validation_data=(X_test_zx, y_test_zx), epochs=EPOCHS)

Epoch 1/30
30/30 [==============================] - 1s 15ms/step - loss: 2.8354 - accuracy: 0.0759 - val_loss: 2.7347 - val_accuracy: 0.1375
Epoch 2/30
30/30 [==============================] - 0s 9ms/step - loss: 2.6344 - accuracy: 0.1433 - val_loss: 2.0579 - val_accuracy: 0.3250
Epoch 3/30
30/30 [==============================] - 0s 9ms/step - loss: 1.9791 - accuracy: 0.3412 - val_loss: 1.7245 - val_accuracy: 0.3750
Epoch 4/30
30/30 [==============================] - 0s 9ms/step - loss: 1.6395 - accuracy: 0.4637 - val_loss: 1.7193 - val_accuracy: 0.3812
Epoch 5/30
30/30 [==============================] - 0s 9ms/step - loss: 1.4100 - accuracy: 0.5315 - val_loss: 1.5583 - val_accuracy: 0.4125
Epoch 6/30
30/30 [==============================] - 0s 9ms/step - loss: 1.1851 - accuracy: 0.6215 - val_loss: 1.6474 - val_accuracy: 0.3500
Epoch 7/30
30/30 [==============================] - 0s 9ms/step - loss: 0.9858 - accuracy: 0.6807 - val_loss: 1.6285 - val_accuracy: 0.3562
Epoch 8/30
30/30 [=

In [439]:
# prob_zx = tf.keras.Sequential([model_zx, tf.keras.layers.Softmax()])
# y_pred_zx = prob_zx.predict(X_test_zx)
y_pred_zx = model_zx.predict(X_test_zx)
y_pred = np.argmax(y_pred_zx, axis=1)
print(classification_report(y_test_zx.ravel(), y_pred, zero_division=0))
prc_zx = precision_score(y_test_zx.ravel(), y_pred, zero_division=0, average=None)
tf.keras.backend.clear_session()

              precision    recall  f1-score   support

         0.0       0.43      0.30      0.35        10
         1.0       0.43      1.00      0.61        10
         2.0       0.33      0.20      0.25        10
         3.0       0.62      0.80      0.70        10
         4.0       0.30      0.30      0.30        10
         5.0       0.50      0.20      0.29        10
         6.0       0.30      0.30      0.30        10
         7.0       0.20      0.20      0.20        10
         8.0       0.45      0.90      0.60        10
         9.0       0.33      0.10      0.15        10
        10.0       0.56      1.00      0.71        10
        11.0       0.00      0.00      0.00        10
        12.0       0.67      0.20      0.31        10
        13.0       0.33      0.20      0.25        10
        14.0       0.67      0.40      0.50        10
        15.0       0.55      0.60      0.57        10

    accuracy                           0.42       160
   macro avg       0.42   

In [440]:
y_total = y_pred_xy + y_pred_yz + y_pred_zx
y_pred = np.argmax(y_total, axis=1)
report = classification_report(y_test_xy.ravel(), y_pred, zero_division=0)
print(report)

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        10
         1.0       1.00      0.50      0.67        10
         2.0       0.60      0.30      0.40        10
         3.0       1.00      1.00      1.00        10
         4.0       0.90      0.90      0.90        10
         5.0       0.67      0.20      0.31        10
         6.0       0.69      0.90      0.78        10
         7.0       0.91      1.00      0.95        10
         8.0       0.75      0.90      0.82        10
         9.0       1.00      1.00      1.00        10
        10.0       0.91      1.00      0.95        10
        11.0       0.31      0.50      0.38        10
        12.0       0.50      0.20      0.29        10
        13.0       0.67      1.00      0.80        10
        14.0       0.88      0.70      0.78        10
        15.0       0.47      0.80      0.59        10

    accuracy                           0.74       160
   macro avg       0.77   

In [441]:
config = '\n\nTEST_USER ' + TEST_USER + ' T: ' + str(int(time.time())) + '\n'
underline = '=====================================\n'
log_dir = os.path.join(BASE_DIR, LOG_DIR)
if not os.path.exists(log_dir):
    os.mkdir(log_dir)
f = open(os.path.join(log_dir, 'logs_splstm_bw' + CONFIG + '.txt'), 'a')
f.write(config)
f.write(underline)
f.write(report)
f.close()

In [442]:
# config = TEST_USER + ' :'
# log_dir = os.path.join(BASE_DIR, LOG_DIR)
# if not os.path.exists(log_dir):
#     os.mkdir(log_dir)
# f = open(os.path.join(log_dir, 'prc_sptl_bw_xy' + CONFIG + '.txt'), 'a')
# f.write(config)
# f.write(np.array2string(prc_xy, precision=2, max_line_width=100) + '\n')
# f.close()

In [443]:
# config = TEST_USER + ' :'
# log_dir = os.path.join(BASE_DIR, LOG_DIR)
# if not os.path.exists(log_dir):
#     os.mkdir(log_dir)
# f = open(os.path.join(log_dir, 'prc_sptl_bw_yz' + CONFIG + '.txt'), 'a')
# f.write(config)
# f.write(np.array2string(prc_yz, precision=2, max_line_width=100) + '\n')
# f.close()

In [444]:
# config = TEST_USER + ' :'
# log_dir = os.path.join(BASE_DIR, LOG_DIR)
# if not os.path.exists(log_dir):
#     os.mkdir(log_dir)
# f = open(os.path.join(log_dir, 'prc_sptl_bw_zx' + CONFIG + '.txt'), 'a')
# f.write(config)
# f.write(np.array2string(prc_zx, precision=2, max_line_width=100) + '\n')
# f.close()